In [ ]:
from typing import List, Callable
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from prettytable import PrettyTable
import torch


def load_zephyr_pipeline():
    model_id = "HuggingFaceH4/zephyr-7b-alpha"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return pipe


def zephyr_model_call(pipe, prompt: str) -> List[str]:
    output = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)[0]["generated_text"]
    cleaned = output.replace(prompt, "").strip()
    values = [v.strip() for v in cleaned.split(",") if v.strip()]
    return values


def fill_table_rows(
    prompt: str,
    columns: List[str],
    rows: List[str],
    model_call_fn: Callable[[str], List[str]]
) -> List[List[str]]:
    full_table = []
    for row_label in rows:
        full_prompt = (
            f"{prompt}\n"
            f"Component: {row_label}\n"
            f"Columns: {', '.join(columns[1:])}\n"
            f"Provide your response as comma-separated values."
        )
        model_response = model_call_fn(full_prompt)
        full_row = [row_label] + model_response
        full_table.append(full_row)
    return full_table


def print_table(columns: List[str], rows: List[List[str]]):
    table = PrettyTable()
    table.field_names = columns
    for row in rows:
        if len(row) == len(columns):
            table.add_row(row)
        else:
            table.add_row(row + [""] * (len(columns) - len(row))) 
    print(table)





Loading Zephyr model... (may take a while)


c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Control\.cache\huggingface\hub\models--HuggingFaceH4--zephyr-7b-alpha. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 8 files:   0%|          | 0/8 [05:39<?, ?it/s]


OSError: HuggingFaceH4/zephyr-7b-alpha does not appear to have files named ('model-00001-of-00008.safetensors', 'model-00002-of-00008.safetensors', 'model-00003-of-00008.safetensors', 'model-00004-of-00008.safetensors', 'model-00005-of-00008.safetensors', 'model-00006-of-00008.safetensors', 'model-00007-of-00008.safetensors', 'model-00008-of-00008.safetensors'). Checkout 'https://huggingface.co/HuggingFaceH4/zephyr-7b-alpha/tree/main'for available files.

In [ ]:

if __name__ == "__main__":
    print("Loading Zephyr model... (may take a while)")
    zephyr_pipe = load_zephyr_pipeline()

    input_data = {
        "prompt": "Given a cleanroom design specification, fill in a table row for the given component.",
        "columns": ["Component", "Requirement", "Justification"],
        "rows": ["Wall Panels", "HVAC", "Cleanroom Flooring"]
    }

    print("Filling table using Zephyr...")
    full_table = fill_table_rows(
        prompt=input_data["prompt"],
        columns=input_data["columns"],
        rows=input_data["rows"],
        model_call_fn=lambda p: zephyr_model_call(zephyr_pipe, p)
    )

    print("\nGenerated Table:\n")
    print_table(input_data["columns"], full_table)

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="GenAI Table Filler")
    parser.add_argument("--prompt", type=str, help="Prompt to guide model table filling", required=False)
    parser.add_argument("--topic", type=str, help="Topic to generate full table from scratch", required=False)
    parser.add_argument("--columns", type=str, nargs="+", help="Column headers (optional)")
    parser.add_argument("--rows", type=str, nargs="+", help="Row names (optional)")
    parser.add_argument("--output", type=str, choices=["console", "pdf", "docx"], default="console")
    parser.add_argument("--intro", type=str, default="This document contains an AI-generated table based on your input.")
    args = parser.parse_args()

    print("Loading model...")
    gpt_neo_pipe = load_gpt_neo_pipeline()

    if args.topic:
        columns, rows = generate_table_structure(gpt_neo_pipe, args.topic)
        prompt = f"Using the topic '{args.topic}', fill in the table rows."
    else:
        prompt = args.prompt or "Fill in the table based on the given prompt and headers."
        columns = args.columns or ["Aspect", "Detail", "Reasoning"]
        rows = args.rows or ["Item 1", "Item 2", "Item 3"]

    print("Generating table content...")
    full_table = fill_table_rows(
        prompt=prompt,
        columns=columns,
        rows=rows,
        model_call_fn=lambda p: model_call(gpt_neo_pipe, p)
    )

    if args.output == "pdf":
        export_table_to_pdf(columns, full_table, args.intro)
    elif args.output == "docx":
        export_table_to_docx(columns, full_table, args.intro)
    else:
        print("\nGenerated Table:\n")
        print_table(columns, full_table)

if __name__ == "__main__":
    main()
